# <center> Applications du Bootstrap : TP2 régression

Antoine Grelety

Vincent Le Meur

In [1]:
import numpy as np
import numpy.random as npr
import pandas as pd
import math
import scipy as sc
from scipy.stats import norm
import statsmodels.genmod.generalized_linear_model as st
from matplotlib import pyplot as plt

## Importation des données

In [2]:
cars = pd.read_csv("mtcars.csv")
cars["intercept"]=np.ones(len(cars))
urine = pd.read_csv("urine.dat",sep="\t")
#urine["intercept"]=np.ones(len(urine))
urine=urine.fillna(np.mean(urine))

In [3]:
cars.head(),urine.head()

(          Unnamed: 0   mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  \
 0          Mazda RX4  21.0    6  160.0  110  3.90  2.620  16.46   0   1     4   
 1      Mazda RX4 Wag  21.0    6  160.0  110  3.90  2.875  17.02   0   1     4   
 2         Datsun 710  22.8    4  108.0   93  3.85  2.320  18.61   1   1     4   
 3     Hornet 4 Drive  21.4    6  258.0  110  3.08  3.215  19.44   1   0     3   
 4  Hornet Sportabout  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3   
 
    carb  intercept  
 0     4        1.0  
 1     4        1.0  
 2     1        1.0  
 3     1        1.0  
 4     2        1.0  ,    r  gravity    ph   osmo       cond  urea  calc
 0  0    1.021  4.91  725.0  20.901282   443  2.45
 1  0    1.017  5.74  577.0  20.000000   296  4.49
 2  0    1.008  7.20  321.0  14.900000   101  2.36
 3  0    1.011  5.51  408.0  12.600000   224  2.15
 4  0    1.005  6.52  187.0   7.500000    91  1.16)

## Définition des fonctions utiles :

In [4]:
# Intervalle de confiance basique 
def IC_base(alpha,beta_hat,beta):
    A= []
    beta=np.sort(np.array(beta),axis=1)
    for k in range(len(beta_hat)):
        B=[]
        B.append( 2*beta_hat[k]-beta[k,math.ceil(len(beta[0])*(1-alpha/2))-1] )
        B.append( 2*beta_hat[k]-beta[k,math.ceil(len(beta[0])*(alpha/2))-1] )        
        A.append(B)
    return(A)

In [5]:
# Intervalle de confiance percentille
def IC_perc(alpha,beta_hat,beta):
    A= []
    beta=np.sort(np.array(beta),axis=1)
    for k in range(len(beta_hat)):
        B=[]
        B.append(beta[k][math.ceil(len(beta[0])*(alpha/2))-1])
        B.append(beta[k][math.ceil(len(beta[0])*(1-alpha/2))-1])
        A.append(B)
    return(A)

In [6]:
# Intervalle de confiance asymptotiquement normal
def ICAN(alpha,X,Y,Y_pred,beta):
    B=[]
    MSE_root=np.sqrt(np.square(Y - Y_pred).mean())
    for d in range(len(beta)-1):
        A= []
        denom=np.sqrt( np.sum( (X[:,d]-np.mean(X[:,d]))**2 ) )
        A.append(beta[d]-MSE_root/denom*norm.ppf(1-alpha/2))
        A.append(beta[d]+MSE_root/denom*norm.ppf(1-alpha/2))
        print(A)
        B.append(A)
    return B

## Estimateurs classiques

### Estimation de Y=mpg dans le dataset mtcars via une régression linéaire

In [7]:
#transformation des données
X_cars=np.array(cars.iloc[:,2:cars.shape[1]])
Y_cars=np.array(cars.iloc[:,1])


In [8]:
Reg_lin = st.GLM(Y_cars,X_cars)
cars_results = Reg_lin.fit()

Les estimateurs des coefficients de la régression linéaire sont : 

In [9]:
# Beta
Beta_cars=cars_results.params
Beta_cars

array([-0.11144048,  0.01333524, -0.02148212,  0.78711097, -3.71530393,
        0.82104075,  0.31776281,  2.52022689,  0.65541302, -0.19941925,
       12.30337416])

Les résidus et leur variance sont données par :

In [10]:
#residus
resid_cars=cars_results.resid_deviance
#Y prédits
y_pred_cars= cars_results.fittedvalues
#variance des résidus
var_cars=(1/(len(cars)-X_cars.shape[1]-1))*sum(resid_cars**2)
resid_cars,var_cars

(array([-1.59950576, -1.11188608, -3.45064408,  0.16259545,  1.00656597,
        -2.28303904, -0.08625625,  1.90398812, -1.6190899 ,  0.50097006,
        -1.39165439,  2.22783789,  1.7004264 , -0.5422247 , -1.63401342,
        -0.53643771,  4.20637064,  4.62709419,  0.50326109,  4.3876309 ,
        -2.14310344, -1.44305322, -2.5321815 , -0.00602198,  2.50832101,
        -0.99346869, -0.15295396,  2.76372742, -3.0700408 ,  0.00617185,
         1.05888162, -2.96826768]), 7.374721500832528)

### Estimation de Y=r dans le dataset urine via une régression logistique

In [11]:
#transformation des données
X_urine=np.array(urine.iloc[:,1:urine.shape[1]])
Y_urine=np.array(urine.iloc[:,0])

In [12]:
Reg_log =st.GLM(Y_urine,X_urine,family=st.families.Binomial())
urine_results = Reg_log.fit()

Les estimateurs des coefficients de la régression logistique sont : 

In [13]:
Beta_urine = urine_results.params
Beta_urine

array([ 0.96106425, -0.38063452,  0.02383801, -0.45313144, -0.02501159,
        0.6669377 ])

Leurs résidus et leur variance sont données par :

In [64]:
#residus
resid_urine=urine_results.resid_pearson
#Y prédits
y_pred_urine= urine_results.fittedvalues
#variance des résidus
var_urine=(1/(len(urine)-X_urine.shape[1]-1))*sum(resid_urine**2)
resid_urine,var_urine

(array([-0.28333126, -0.62989078, -0.40178604, -0.52784935, -0.37548893,
        -0.7256254 , -0.3669093 , -1.53054825, -0.46263933, -0.28533154,
        -0.64211937, -0.2559101 , -0.36628807, -0.47124532, -0.49612119,
        -0.94324571, -0.37580687, -0.31188909, -0.51820996, -0.22611485,
        -0.90888037, -0.2542608 , -0.2214696 , -0.65615327, -0.48943593,
        -0.61368818, -0.47806868, -0.31507163, -0.53396061, -0.31423202,
        -0.1621946 , -1.40103534, -1.16524409, -1.1290385 , -0.50510753,
        -0.69310826, -0.30112818, -1.02433397, -0.64783216, -1.34620691,
        -0.53355578, -0.40992513, -0.57707548, -0.66762016, -0.54043874,
         0.5940306 ,  0.04062338,  0.66379058,  1.11185648,  0.28049224,
         0.01185032,  1.03147979,  1.40628922,  4.68284913,  5.00671709,
         0.79179205,  0.65010047,  0.40924191,  0.06482248,  1.10087188,
         0.35100158,  0.64931068,  0.01087765,  0.06421688,  0.6847924 ,
         1.40238041,  0.35113631,  1.66316156,  2.0

## Intervalles de confiance par bootstrap

### Cases sampling

#### Dataset cars :

In [15]:
beta_CS=[]
B=2000
# Mise en place du Bootstrap
for b in range(1,B):
    X_cars_sample=np.array(cars.iloc[:,2:cars.shape[1]])
    Y_cars_sample=np.array(cars.iloc[:,1])
    ind= np.random.randint(0,len(cars),len(cars))
    sample_X=X_cars_sample[ind,:]
    sample_Y=Y_cars_sample[ind]
    lin_model_cars_sample =st.GLM(sample_Y,sample_X,st.families.Gaussian())
    lin_cars_res_sample = lin_model_cars_sample.fit()
    beta_cars_sample = lin_cars_res_sample.params
    beta_CS.append(beta_cars_sample)
beta_CS

[array([ 5.16786465e-02,  1.66179630e-02,  1.65661780e-02,  3.06537340e+00,
        -7.73925640e+00,  4.47557062e+00, -5.30393509e+00, -1.21706014e+00,
         1.59425284e+00,  7.51012283e-01, -5.68989133e+01]),
 array([ 0.9818052 ,  0.03671403, -0.07181169,  0.75797295, -9.0368814 ,
         2.0005803 , -0.80523087,  4.97316872, -1.7349081 ,  2.20170927,
         5.65674955]),
 array([-0.61076758,  0.0308066 , -0.03174657,  0.45323556, -5.6218801 ,
         1.47325522, -1.52801593,  2.3537368 ,  0.15162067,  0.72127017,
         8.35119147]),
 array([-2.16919167e+00, -1.67207643e-02, -3.95801545e-02, -9.57533878e-01,
         2.44229747e+00, -3.22491851e+00,  4.75486448e-01, -9.24152277e-01,
         8.85046682e-01, -2.47653179e+00,  9.95809404e+01]),
 array([-0.13676424,  0.00723998, -0.01627651,  3.02342366, -3.35515184,
         1.09926057, -2.19759223,  2.0918454 , -1.63922747,  0.23198151,
         6.4907514 ]),
 array([-1.99126634e+00, -2.23968558e-02, -2.24162339e-02,  3.64735

##### Intervalles de confiance :

In [16]:
alpha=0.05

Les intervalles de confiance version bootstrap basiques sont :

In [17]:
IC_base(alpha,Beta_cars,beta_CS)

[[-4.698451570807459, 56.676032336784175],
 [-5.630079067053837, 9.063551876236573],
 [-8.394155703918692, 5.578915859337738],
 [-98.0067185046822, 4.799140454426943],
 [-13.921359252235206, -4.075456014374158],
 [-94.76284097499679, 5.0087382327611465],
 [-10.048810610835403, 5.727204331800377],
 [0.8116133471532461, 8.56721518347032],
 [-19.471032965094853, 4.27803795161648],
 [-4.976908194822424, 33.007195299883044],
 [-3.695542527603365, 31.595989837959202]]

Les intervalles de confiance version percentile sont :

In [18]:
IC_perc(alpha,Beta_cars,beta_CS)

[[-56.89891329255789, 4.475570615033744],
 [-9.03688139640989, 5.656749546880519],
 [-5.621880097316012, 8.351191465940417],
 [-3.224918509954704, 99.58094044915444],
 [-3.355151842280822, 6.490751395580226],
 [-3.366656733411892, 96.40492247434605],
 [-5.091678703429534, 10.684336239206246],
 [-3.5267614090534636, 4.228840427263611],
 [-2.9672119174529143, 20.781858999258418],
 [-33.40603380959557, 4.5780696851099005],
 [-6.989241525966614, 28.302290839595955]]

Les intervalles de confiance version Asymptotique normale sont :

In [19]:
ICAN(alpha,X_cars,Y_cars,y_pred_cars,Beta_cars)

[-0.5346131984761235, 0.3117322427024091]
[0.007237440277612639, 0.01943303954906908]
[-0.03250489762645949, -0.010459340351814573]
[-0.6263608401108678, 2.2005827845831067]
[-4.4876965627873115, -2.9429112938676685]
[0.39810995543832145, 1.2439715439109331]
[-1.1816997604264183, 1.8172253887972611]
[1.005663603603752, 4.034790170813105]
[-0.36891492808131987, 1.6797409622448858]
[-0.6673200303414686, 0.26848152062894526]


[[-0.5346131984761235, 0.3117322427024091],
 [0.007237440277612639, 0.01943303954906908],
 [-0.03250489762645949, -0.010459340351814573],
 [-0.6263608401108678, 2.2005827845831067],
 [-4.4876965627873115, -2.9429112938676685],
 [0.39810995543832145, 1.2439715439109331],
 [-1.1816997604264183, 1.8172253887972611],
 [1.005663603603752, 4.034790170813105],
 [-0.36891492808131987, 1.6797409622448858],
 [-0.6673200303414686, 0.26848152062894526]]

#### Dataset urine : 

In [20]:
Beta_CSU=[]
B=2000
# Mise en place du bootstrap
for b in range(1,B):
    X_urine_sample=np.array(urine.iloc[:,1:urine.shape[1]])
    Y_urine_sample=np.array(urine.iloc[:,0])
    ind= np.random.randint(0,len(urine),len(urine))
    sample_X=X_urine_sample[ind,:]
    sample_Y=Y_urine_sample[ind]
    lin_model_cars_sample =st.GLM(sample_Y,sample_X,st.families.Binomial())
    lin_cars_res_sample = lin_model_cars_sample.fit()
    beta_urine_sample = lin_cars_res_sample.params
    Beta_CSU.append(beta_urine_sample)
Beta_CSU

[array([10.23544383, -2.04309397,  0.03666186, -0.64438985, -0.03754584,
         0.80766001]),
 array([-4.79889006,  0.51810096,  0.04606467, -0.76056253, -0.05052419,
         0.55350112]),
 array([ 1.42348488, -0.35506537,  0.04898129, -0.84752324, -0.05487015,
         0.69939745]),
 array([-0.15103749, -0.10405538,  0.00930135, -0.24289138, -0.00863536,
         0.51910223]),
 array([-2.26591323,  0.13575658,  0.01640845, -0.33697774, -0.01440276,
         0.46873551]),
 array([ 2.79437328, -0.38689912,  0.02118866, -0.43906131, -0.02441466,
         0.45434177]),
 array([-1.88215714, -0.22422035,  0.0827286 , -1.35633076, -0.08843235,
         1.16723761]),
 array([ 1.25563436, -0.47129932,  0.02501406, -0.49523406, -0.02731456,
         0.90128132]),
 array([-5.757157  ,  0.80380111,  0.03515755, -0.58889077, -0.03832921,
         0.47017733]),
 array([ 0.51570499, -0.19028038,  0.00797739, -0.21171232, -0.01287824,
         0.92881561]),
 array([ 5.98755490e-01, -4.31602339e-01

##### Intervalles de confiance :

In [21]:
alpha=0.05

Les intervalles de confiance version bootstrap basique sont : 

In [22]:
IC_base(alpha,Beta_urine,Beta_CSU)

[[-8.313315326086157, 3.9652224768545716],
 [-1.3147701743261724, 4.037621008210877],
 [-1.3758088637513897, 0.895199260130783],
 [-1.4253651128539977, -0.6633715037847356],
 [-0.518758679033932, 2.215890051275253],
 [-1.4604978711700083, 1.7729367099462743]]

Les intervalles de confiance version percentile sont :

In [23]:
IC_perc(alpha,Beta_urine,Beta_CSU)

[[-2.0430939689617977, 10.23544383397893],
 [-4.798890057673901, 0.5535011248631483],
 [-0.8475232446865206, 1.423484879195652],
 [-0.24289138390610582, 0.5191022251631563],
 [-2.2659132250184375, 0.46873550529074715],
 [-0.4390613052460397, 2.794373275870243]]

Les intervalles de confiance version asymptotique normal sont :

In [24]:
ICAN(alpha,X_urine,Y_urine,y_pred_urine,Beta_urine)

[-10.350878633716924, 12.273007141609696]
[-0.49369772481842766, -0.26757132464459643]
[0.02349205460339506, 0.024183960840867262]
[-0.46349635594449023, -0.44276653174635117]
[-0.025635507915316276, -0.02438766582786856]


[[-10.350878633716924, 12.273007141609696],
 [-0.49369772481842766, -0.26757132464459643],
 [0.02349205460339506, 0.024183960840867262],
 [-0.46349635594449023, -0.44276653174635117],
 [-0.025635507915316276, -0.02438766582786856]]

### Error sampling

#### Dataset cars :

In [25]:
#tirage aléatoire avec remise

list_y_sample=list()
Beta_ES=list()
res_ES=list()

for b in range(B):
    y_cars_sample=np.zeros(len(cars))
    ind=npr.randint(0,len(cars),len(cars))
    sample_X=X_cars[ind,:]
    sample_Y=Y_cars[ind]
    sample_resid=resid_cars[ind]
    for i in range(len(cars)):
        y_cars_sample[i] = np.dot(sample_X[i,:],Beta_cars) + np.sqrt(var_cars)*sample_resid[i]
    list_y_sample.append(y_cars_sample)
    #calcule des Beta et variances avec une regression linéaire sur l'échantillon bootstrapé
    lin_model_cars_sample =st.GLM(sample_Y,sample_X,st.families.Gaussian())
    lin_cars_res_sample = lin_model_cars_sample.fit()
    beta_cars_sample = lin_cars_res_sample.params
    resid_cars_sample =lin_cars_res_sample.resid_deviance
    sd_sample=np.std(resid_cars_sample)
    #sauvegarde dans une liste les beta's et erreurs estimés
    Beta_ES.append(beta_cars_sample)
    res_ES.append(sd_sample)
Beta_ES

[array([ 3.23698048e+00, -1.15755608e-02,  1.34653848e-01,  5.40146658e+00,
        -1.58503575e+00,  4.05074579e+00,  2.69444697e-01, -1.91724579e+00,
         1.12204496e+01, -6.60408342e+00, -1.26024647e+02]),
 array([ 0.77686542, -0.0069897 ,  0.02703677,  3.93877984,  0.7174717 ,
        -0.11980409,  4.69172234,  4.17418789,  2.85712012, -3.13226548,
        -6.20964832]),
 array([-1.39119865e-01,  5.12230223e-03, -1.26360028e-02,  1.06922785e+00,
        -2.03790937e+00,  4.22772595e-01,  1.36405662e+00,  3.42628575e+00,
         5.69897677e-01, -7.91235321e-01,  1.45736531e+01]),
 array([ 0.92795891,  0.02244867, -0.0409169 , -1.77557665, -6.01833347,
         1.97094942, -0.02835421,  5.06728276,  1.72217801,  0.31342748,
        -3.69852929]),
 array([  1.30821354,   0.0394654 ,  -0.0170158 ,   1.70117434,
         -6.94679846,   1.48395531,   1.0119678 ,   3.17203355,
          2.53134624,  -0.72271028, -14.28389825]),
 array([ 4.59492751e+00, -2.07740450e-02,  2.06853540e-0

##### Intervalles de confiance :

In [26]:
alpha=0.05

Les intervalles de confiance version bootstrap basique sont :

In [27]:
IC_base(alpha,Beta_cars,Beta_ES)

[[-11.44333057957472, 125.80176614814255],
 [-4.665051863752969, 6.236318798647933],
 [-14.616617320093859, 1.9949451313811668],
 [-3.4930608124999623, 7.592555411160884],
 [-10.602641404873257, 6.853290389493708],
 [-2.9528460090720925, 33.767217612747785],
 [-12.035692854940322, 2.0766659831217114],
 [-44.836730897162624, 9.543824536447579],
 [-3.4015009269335943, 29.27716345521632],
 [-6.544783171038694, 30.88322224247295],
 [14.65209449505494, 28.728362795044255]]

Les intervalles de confiance version percentile sont :

In [28]:
IC_perc(alpha,Beta_cars,Beta_ES)

[[-126.02464710391627, 11.220449623801006],
 [-6.209648318821252, 4.6917223435796505],
 [-2.037909369359441, 14.573653082115584],
 [-6.018333466688645, 5.067282756972201],
 [-14.283898246148688, 3.172033548218277],
 [-32.12513611339853, 4.594927508421347],
 [-1.4411403547508685, 12.671218483311165],
 [-4.5033707620307215, 49.877184671579485],
 [-27.966337421052756, 4.71232696109716],
 [-31.282060752185473, 6.145944661326171],
 [-4.1216144830516654, 9.95465381693765]]

Les intervalles de confiance version asymptotique normale sont :

In [29]:
ICAN(alpha,X_cars,Y_cars,y_pred_cars,Beta_cars)

[-0.5346131984761235, 0.3117322427024091]
[0.007237440277612639, 0.01943303954906908]
[-0.03250489762645949, -0.010459340351814573]
[-0.6263608401108678, 2.2005827845831067]
[-4.4876965627873115, -2.9429112938676685]
[0.39810995543832145, 1.2439715439109331]
[-1.1816997604264183, 1.8172253887972611]
[1.005663603603752, 4.034790170813105]
[-0.36891492808131987, 1.6797409622448858]
[-0.6673200303414686, 0.26848152062894526]


[[-0.5346131984761235, 0.3117322427024091],
 [0.007237440277612639, 0.01943303954906908],
 [-0.03250489762645949, -0.010459340351814573],
 [-0.6263608401108678, 2.2005827845831067],
 [-4.4876965627873115, -2.9429112938676685],
 [0.39810995543832145, 1.2439715439109331],
 [-1.1816997604264183, 1.8172253887972611],
 [1.005663603603752, 4.034790170813105],
 [-0.36891492808131987, 1.6797409622448858],
 [-0.6673200303414686, 0.26848152062894526]]

#### Dataset urine :

In [37]:
#tirage aléatoire avec remise

list_y_sample=list()
Beta_ESU=list()
res_ESU=list()

for b in range(B):
    y_urine_sample=np.zeros(len(urine))
    ind=npr.randint(0,len(urine),len(urine))
    sample_X=X_urine[ind,:]
    sample_Y=Y_urine[ind]
    sample_resid=resid_urine[ind]
    for i in range(len(urine)):
        y_urine_sample[i] = np.dot(sample_X[i,:],Beta_urine) + np.sqrt(var_urine)*sample_resid[i]
    list_y_sample.append(y_urine_sample)
    #calcule des Beta et variances avec une regression linéaire sur l'échantillon bootstrapé
    lin_model_urine_sample =st.GLM(sample_Y,sample_X,st.families.Binomial())
    lin_urine_res_sample = lin_model_urine_sample.fit()
    beta_urine_sample = lin_urine_res_sample.params
    resid_urine_sample =lin_urine_res_sample.resid_pearson
    sd_sample=np.std(resid_urine_sample)
    #sauvegarde dans une liste les beta's et erreurs estimés
    Beta_ESU.append(beta_urine_sample)
    res_ESU.append(sd_sample)
Beta_ESU

[array([15.16932828, -2.70230317,  0.0809838 , -1.48623456, -0.08856363,
         1.21970169]),
 array([ 0.1631466 , -0.35373767,  0.04925356, -0.83292389, -0.05683482,
         0.93707398]),
 array([-3.89650715,  0.36804392,  0.02135576, -0.34728265, -0.02526125,
         0.58514016]),
 array([ 7.26539299, -1.44868456,  0.05402591, -0.96078699, -0.06567274,
         1.192609  ]),
 array([-0.67846747, -0.153252  ,  0.01269497, -0.26555961, -0.01010205,
         0.49261954]),
 array([ 4.57832406, -1.10798446,  0.03019833, -0.61354427, -0.02646336,
         0.73720907]),
 array([-0.4596773 , -0.03020522,  0.02122975, -0.46106349, -0.01818931,
         0.52927526]),
 array([-0.86553879, -0.11842199,  0.01794564, -0.45299761, -0.01140195,
         0.62642477]),
 array([-4.96624089e+00,  4.07727446e-01,  8.25149054e-03, -2.21311270e-01,
        -4.02217871e-03,  7.81515591e-01]),
 array([-1.99229854,  0.22010053,  0.01882024, -0.41205246, -0.01396522,
         0.43100594]),
 array([ 2.07856

##### Intervalles de confiance :

In [31]:
alpha=0.05

Les intervalles de confiance version bootstrap basique sont :

In [38]:
IC_base(alpha,Beta_urine,Beta_ESU)

[[-13.24719976930547, 4.624431682350742],
 [-1.6983430304246123, 0.07165483619832269],
 [-0.5374641456230003, 3.9441831681194657],
 [-8.171655876742019, 0.5424216673609425],
 [-0.542642708860934, 0.6284443003107085],
 [-3.2444486550705958, 2.441859861502052]]

Les intervalles de confiance version percentile sont :

In [39]:
IC_perc(alpha,Beta_urine,Beta_ESU)

[[-2.7023031744579677, 15.169328277198243],
 [-0.8329238856613468, 0.9370739809615882],
 [-3.8965071526752033, 0.5851401610672626],
 [-1.448684555051784, 7.265392989051177],
 [-0.6784674740538934, 0.4926195351177492],
 [-1.1079844568018171, 4.57832405977083]]

Les intervalles de confiance version asymptotique normale sont :

In [40]:
ICAN(alpha,X_urine,Y_urine,y_pred_urine,Beta_urine)

[-10.350878633716924, 12.273007141609696]
[-0.49369772481842766, -0.26757132464459643]
[0.02349205460339506, 0.024183960840867262]
[-0.46349635594449023, -0.44276653174635117]
[-0.025635507915316276, -0.02438766582786856]


[[-10.350878633716924, 12.273007141609696],
 [-0.49369772481842766, -0.26757132464459643],
 [0.02349205460339506, 0.024183960840867262],
 [-0.46349635594449023, -0.44276653174635117],
 [-0.025635507915316276, -0.02438766582786856]]

SCHEMA COMPARATIFS A GLISSER ICI

## Tests par Bootstrap

### Dataset cars

In [100]:
# Cette fonction permet de réaliser une régression linéaire ou logistique sur l'un des deux datasets (variable choice)
# En ayant préalablement retiré la colonne d'indice k (dans le but de faire un test de nullité)
# Elle retourne les nouvelles estimations de Beta ainsi que les résidus associés
def Estim_ES(data,choice,k):
    list_y_sample=list()
    Beta_ES=list()
    res_ES=list()
    if choice=="cars":
        if k==2:
            df=data.drop(data.columns[k],1)
            X=np.array(df.iloc[:,3:df.shape[1]])
            Y=np.array(df.iloc[:,1])
            model=st.families.Gaussian()
            Reg_lin = st.GLM(Y,X,model)
            results = Reg_lin.fit()
            Beta=results.params
            resid=results.resid_deviance
        else:
            df=data.drop(data.columns[k],1)
            X=np.array(df.iloc[:,2:df.shape[1]])
            Y=np.array(df.iloc[:,1])
            model=st.families.Gaussian()
            Reg_lin = st.GLM(Y,X,model)
            results = Reg_lin.fit()
            Beta=results.params
            resid=results.resid_deviance
    if choice=="urine":
        if k==1:
            df=data.drop(data.columns[k],1)
            X=np.array(data.iloc[:,2:data.shape[1]])
            Y=np.array(data.iloc[:,0])
            model=st.families.Binomial()
            Reg_lin = st.GLM(Y,X,model)
            results = Reg_lin.fit()
            Beta=results.params
            resid=results.resid_pearson
        else:
            df=data.drop(data.columns[k],1)
            X=np.array(data.iloc[:,1:data.shape[1]])
            Y=np.array(data.iloc[:,0])
            model=st.families.Binomial()
            Reg_lin = st.GLM(Y,X,model)
            results = Reg_lin.fit()
            Beta=results.params
            resid=results.resid_pearson
    return(Beta,resid)

In [104]:
Estim_ES(cars,"cars",3)[1]

array([-1.3758673 , -1.07852791, -3.78630744,  0.57714515,  1.43948317,
       -2.365721  ,  0.25044175,  2.01727087, -1.40322882,  0.22243718,
       -1.5767032 ,  1.29756442,  1.17135708, -1.06338937, -0.91786827,
       -0.28984949,  4.04937573,  4.36312671,  1.1045535 ,  4.47528278,
       -2.22381839, -1.41215224, -2.58366206, -0.2371252 ,  3.04844269,
       -1.05382976, -0.31885869,  3.03844697, -3.06561165, -0.05072927,
        1.13697662, -3.38865455])

In [107]:
Residus = Estim_ES(cars,"cars",3)[1]
Sample_Result=[]
for b in range(B):
        ind=npr.randint(0,len(Residus),len(Residus))
        sample_res=[]
        for k in ind:
            sample_res.append(Residus[k])
        Sample_Result.append(sample_res)   
Sample_Result        

[[-3.388654552098366,
  -3.388654552098366,
  4.049375731855669,
  -0.917868267670249,
  -0.3188586922127712,
  4.363126714053195,
  1.439483172766355,
  -1.0538297606658027,
  -2.583662063779279,
  0.22243718172159532,
  -1.0785279138996628,
  1.1045535000565572,
  1.1045535000565572,
  -0.3188586922127712,
  4.363126714053195,
  0.22243718172159532,
  1.297564420700482,
  1.1045535000565572,
  2.017270871715766,
  -1.3758673005093343,
  -0.3188586922127712,
  -1.0538297606658027,
  -0.2898494908767173,
  -2.3657209997989064,
  -1.3758673005093343,
  -0.3188586922127712,
  1.439483172766355,
  -3.0656116545072187,
  -1.3758673005093343,
  0.2504417489357209,
  -3.786307435722822,
  0.2504417489357209],
 [-2.583662063779279,
  4.475282780755165,
  -0.23712519800793075,
  4.363126714053195,
  1.1045535000565572,
  -2.2238183893960652,
  -1.3758673005093343,
  2.017270871715766,
  3.038446965521011,
  -1.0538297606658027,
  4.363126714053195,
  -2.2238183893960652,
  0.22243718172159532,

In [ ]:
for b in range(B):
    y_cars_sample=np.zeros(len(cars))
    
    for i in range(len(cars)):
        y_cars_sample[i] = np.dot(sample_X[i,:],Beta_cars) + np.sqrt(var_cars)*sample_resid[i]
    list_y_sample.append(y_cars_sample)
    #calcule des Beta et variances avec une regression linéaire sur l'échantillon bootstrapé
    lin_model_cars_sample =st.GLM(sample_Y,sample_X,st.families.Gaussian())
    lin_cars_res_sample = lin_model_cars_sample.fit()
    beta_cars_sample = lin_cars_res_sample.params
    resid_cars_sample =lin_cars_res_sample.resid_deviance
    sd_sample=np.std(resid_cars_sample)
    #sauvegarde dans une liste les beta's et erreurs estimés
    Beta_ES.append(beta_cars_sample)
    res_ES.append(sd_sample)
Beta_ES

## Sur simulations

In [53]:
# Définition des paramètres et des échantillons
n=50
p=5
beta_0=2
beta_1=2
beta_2=2
beta_3=0
beta_4=0
beta_5=0

X=np.zeros((n,p))
for k in range(n):
    for j in range(p):
        X[k][j]=npr.uniform(-0.5,0.5)
Y=np.zeros((n,1))
for k in range(n):
    Y[k]=beta_0+beta_1*X[k][0]+beta_2*X[k][1]+beta_3*X[k][2]+beta_4*X[k][3]+beta_5*beta_1*X[k][4]+npr.normal(0,1)